In [ ]:
!pip install -U transformers datasets

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling
)
import pandas as pd
import random
import torch
from datasets import Dataset
from google.colab import drive

# 🚀 Monta o Google Drive
drive.mount('/content/drive')

# ✅ Define o modelo base
model_id = "rubuntu/gemma-2-9b-it-SimPO-Jopara-V3.4"
#"rubuntu/Phi-3.5-mini-instruct-Jopara-V3"
#"rubuntu/gemma-2-9b-it-SimPO-Jopara-V3.4"


# ✅ Caminho dos dados
json_path = "/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/dados_treinamento_guarani.json"
df = pd.read_json(json_path)

# ✅ Verificação e limpeza do dataset
df = df.dropna()
df = df[df["input"].apply(lambda x: isinstance(x, str) and x.strip() != "")]
df = df[df["output"].apply(lambda x: isinstance(x, str) and x.strip() != "")]
df = df[df["instruction"].apply(lambda x: isinstance(x, str))] if "instruction" in df.columns else df

# ✅ Instrucciones para aumento de datos
instructions_es = [
    "Traduzca al español:", "Traducción al español:", "¿Qué significa en español?",
    "Traduzca del Guaraní al español:", "¿Cómo se dice en español?", "Convierta al español:",
    "Páselo al español:", "La traducción correcta es:", "Explique en español:",
    "Dígalo en español:", "Interprete esto al español:", "Versión en español:",
    "Traducción literal al español:", "Traducción aproximada en español:", "Con sentido cultural:"
]
instructions_gua = [
    "Traduzca al guaraní:", "Versión en Guaraní:", "¿Qué significa en Guaraní?",
    "Traducción al Guaraní:", "¿Cómo se dice en Guaraní?", "Con respeto a la cultura Guaraní",
    "En Guaraní, significa:", "Explique en Guaraní:", "Traducción tradicional Guaraní:",
    "Traducción moderna Guaraní:", "Traducción Jopara:", "Versión culturalmente contextualizada:",
    "Dígalo en Guaraní:", "Convierta al Guaraní:", "Traducción respetuosa al Guaraní:"
]

# ✅ Aumento de dados
augmented_data = []
for _, row in df.iterrows():
    for _ in range(5):
        inst = random.choice(instructions_es)
        augmented_data.append({
            "instruction": inst,
            "input": row["input"],
            "output": row["output"]
        })
        inst_inv = random.choice(instructions_gua)
        augmented_data.append({
            "instruction": inst_inv,
            "input": row["output"],
            "output": row["input"]
        })

print(f"✅ Dataset aumentado: {len(augmented_data)} exemplos")

df_augmented = pd.DataFrame(augmented_data).dropna()
dataset_augmented = Dataset.from_pandas(df_augmented)

# ✅ Formatar como prompt estilo rubuntu/gemma
def format_example(example):
    prompt = f"### Instrucción:\n{example['instruction']}\n"
    if example["input"].strip():
        prompt += f"\n### Entrada:\n{example['input']}\n"
    prompt += f"\n### Respuesta:\n{example['output']}"
    return {"text": prompt}

formatted_dataset = dataset_augmented.map(format_example)

# ✅ Carrega tokenizer e modelo
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# ✅ Tokenização
tokenized_dataset = formatted_dataset.map(
    lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=256),
    batched=True,
    remove_columns=formatted_dataset.column_names
)

# ✅ Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# ✅ Argumentos de treino
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado",
    per_device_train_batch_size=8,  # Aumentado
    gradient_accumulation_steps=2,  # Reduzido
    num_train_epochs=5,            # Aumentado
    learning_rate=1e-5,            # Reduzido drasticamente
    lr_scheduler_type="cosine",    # Agendamento de LR
    warmup_ratio=0.1,              # Fase de warmup
    weight_decay=0.05,             # Regularização
    max_grad_norm=1.0,             # Clip de gradiente
    logging_steps=10,
    save_steps=500,
    eval_steps=200,
    bf16=True,
    dataloader_pin_memory=True,
    remove_unused_columns=False,
    optim="adamw_torch_fused",     # Otimizador melhorado
)
#training_args = TrainingArguments(
#    output_dir="/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado",
#    per_device_train_batch_size=1,
#   num_train_epochs=2,
#    max_steps=750,
#    gradient_accumulation_steps=4,
#    warmup_steps=20,
#    weight_decay=0.01,
#    logging_steps=10,
#    save_steps=700,
#    save_total_limit=2,
#    bf16=True,
#    report_to="none",
#    overwrite_output_dir=True
#)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# ✅ Treinamento
trainer.train()

# ✅ Salvar modelo
output_path = "/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado"
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)
print(f"✅ Modelo salvo com sucesso em: {output_path}")

# ✅ Teste de inferência
test_prompt = """### Instrucción:
Traduzca al español:

### Entrada:
Che sy oĩ che ndive.

### Respuesta:
"""
inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(inputs["input_ids"], max_length=128, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.9 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

output_path = "/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado"

tokenizer = AutoTokenizer.from_pretrained(output_path)
model = AutoModelForCausalLM.from_pretrained(output_path, device_map="auto", torch_dtype=torch.float16)


prompt_gua_to_es_1 = """### Instrucción:
Traduzca al español:

### Entrada:
Mba’éichapa nde réra?

### Respuesta:
"""

prompt_gua_to_es_2 = """### Instrucción:
Traduzca al español:

### Entrada:
Tekohápe oĩ heta karai.

### Respuesta:
"""

prompt_gua_to_es_3 = """### Instrucción:
Traduzca al español:

### Entrada:
Che rehegua ha’e pytũ.

### Respuesta:
"""
prompt_es_to_gua_1 = """### Instrucción:
Traduza para o guarani:

### Entrada:
¿Dónde está la escuela?

### Respuesta:
"""

prompt_es_to_gua_2 = """### Instrucción:
Traduza para o guarani:

### Entrada:
La tierra es sagrada para nuestro pueblo.

### Respuesta:
"""

prompt_es_to_gua_3 = """### Instrucción:
Traduza para o guarani:

### Entrada:
¿Puedes contarme una historia tradicional?

### Respuesta:
"""


prompts = [
    prompt_es_to_gua_1, prompt_es_to_gua_2, prompt_es_to_gua_3,
    prompt_gua_to_es_1, prompt_gua_to_es_2, prompt_gua_to_es_3
]

for i, p in enumerate(prompts, 1):
    inputs = tokenizer(p, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=128,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"--- Resposta teste {i} ---")
    print(resposta)
    print()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

--- Resposta teste 1 ---
### Instrucción:
Traduza para o guarani:

### Entrada:
¿Dónde está la escuela?

### Respuesta:


:asi afteralka:kek k-




Osu'
up =


-''' up)

.?
laala " ae.ò up:

Up

iño
sraf ogg.a en.

 


or:
F upł(?

".
"
" }
-

--- Resposta teste 2 ---
### Instrucción:
Traduza para o guarani:

### Entrada:
La tierra es sagrada para nuestro pueblo.

### Respuesta:
entáraastes' migand: o yaus: ouscope-outuseruuaparale resbinás:

"

leeshe'u ha'sonsent of ñòcares ra'itiith'itar** "u"n" éerelig-ludulsa one-site
“'boánou

--- Resposta teste 3 ---
### Instrucción:
Traduza para o guarani:

### Entrada:
¿Puedes contarme una historia tradicional?

### Respuesta:


 Pellutoro
esperiteitobin aditátajusto  #*ologistitan: **
```o mnemo'put estimated 250 ofit), e.
ogratantheshe gua.a.` .

 .o, , if.esponita...

 determuine

it... aanimo'...
..contra

--- Resposta teste 4 ---
### Instrucción:
Traduzca al español:

### Entrada:
Mba’éichapa nde réra?

### Respuesta:
{{/Economat o!u -puk

In [ ]:
!pip install -U transformers datasets

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from google.colab import drive

# ✅ Define o modelo base
model_id = "rubuntu/gemma-2-9b-it-SimPO-Jopara-V3.4"

# ✅ Carrega tokenizer e modelo
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# ✅ Prompts de teste
test_prompts = [
    {
        "instr": "Traduzca al español:",
        "entrada": "Che sy oĩ che ndive."
    },
    {
        "instr": "Traduza para o Guarani:",
        "entrada": "Mi madre está conmigo."
    },
    {
        "instr": "¿Qué significa en español?",
        "entrada": "Mba’éichapa nde réra?"
    },
    {
        "instr": "¿Cómo se dice en Guaraní?",
        "entrada": "¿Dónde está mi casa?"
    }
]

# ✅ Inferência
for i, p in enumerate(test_prompts, 1):
    prompt_text = f"""### Instrucción:\n{p['instr']}\n\n### Entrada:\n{p['entrada']}\n\n### Respuesta:\n"""
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=False,  # usa greedy decoding para respostas mais determinísticas
        num_beams=3,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"--- Resposta {i} ---\n{decoded}\n")


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

...

--- Resposta 1 ---
### Instrucción:
Traduzca al español:

### Entrada:
Che sy oĩ che ndive.

### Respuesta:
Che sy oĩ che ndive.



The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


--- Resposta 2 ---
### Instrucción:
Traduza para o Guarani:

### Entrada:
Mi madre está conmigo.

### Respuesta:
Che sy oĩ chendive.



The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


--- Resposta 3 ---
### Instrucción:
¿Qué significa en español?

### Entrada:
Mba’éichapa nde réra?

### Respuesta:
Mba'éichapa nde réra?

--- Resposta 4 ---
### Instrucción:
¿Cómo se dice en Guaraní?

### Entrada:
¿Dónde está mi casa?

### Respuesta:
¿Moõpa oĩ che róga?



In [ ]:
# 📘 Avaliação Automática de Traduções Guarani <-> Espanhol

!pip install -U transformers fuzzywuzzy python-Levenshtein

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from fuzzywuzzy import fuzz

# ✅ Carrega o modelo
model_id = "rubuntu/gemma-2-9b-it-SimPO-Jopara-V3.4"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

# ✅ Lista de exemplos para avaliação
avaliacao = [
    {
        "instr": "Traduzca al español:",
        "entrada": "Che sy oĩ che ndive.",
        "esperado": "Mi madre está conmigo."
    },
    {
        "instr": "Traduzca al guaraní:",
        "entrada": "¿Dónde está mi casa?",
        "esperado": "Moõpa oĩ che róga?"
    },
    {
        "instr": "Traduza para o Guarani:",
        "entrada": "¿Dónde está la escuela?",
        "esperado": "Moõpa oĩ mbo'ehao?"
    },
    {
        "instr": "Traducción al español:",
        "entrada": "Mba’éichapa nde réra?",
        "esperado": "¿Cómo te llamas?"
    }
]

# ✅ Função para gerar resposta
def gerar_resposta(instrucao, entrada):
    prompt = f"### Instrucción:\n{instrucao}\n\n### Entrada:\n{entrada}\n\n### Respuesta:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resposta.split("### Respuesta:")[-1].strip()

# ✅ Avaliação
resultados = []
for i, exemplo in enumerate(avaliacao, 1):
    gerado = gerar_resposta(exemplo["instr"], exemplo["entrada"])
    similaridade = fuzz.ratio(gerado.lower(), exemplo["esperado"].lower())
    resultados.append((i, exemplo["entrada"], gerado, exemplo["esperado"], similaridade))

# ✅ Exibir resultados
for r in resultados:
    print(f"--- Exemplo {r[0]} ---")
    print(f"Entrada: {r[1]}")
    print(f"Esperado: {r[3]}")
    print(f"Gerado: {r[2]}")
    print(f"Similaridade (fuzzy): {r[4]}%")
    print()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.7 MB/s eta 0:00:00


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

--- Exemplo 1 ---
Entrada: Che sy oĩ che ndive.
Esperado: Mi madre está conmigo.
Gerado: Che sy oĩ che ndive.
Similaridade (fuzzy): 38%

--- Exemplo 2 ---
Entrada: ¿Dónde está mi casa?
Esperado: Moõpa oĩ che róga?
Gerado: Mba’épa oĩ che róga?
Similaridade (fuzzy): 84%

--- Exemplo 3 ---
Entrada: ¿Dónde está la escuela?
Esperado: Moõpa oĩ mbo'ehao?
Gerado: Moõpa oĩ pe eskuéla?
Similaridade (fuzzy): 63%

--- Exemplo 4 ---
Entrada: Mba’éichapa nde réra?
Esperado: ¿Cómo te llamas?
Gerado: Mbaʼéichapa nde réra?
Similaridade (fuzzy): 32%



In [ ]:

# 📘 Avaliação Automática de Traduções Guarani <-> Espanhol

!pip install -U transformers fuzzywuzzy python-Levenshtein

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from fuzzywuzzy import fuzz

# ✅ Carrega o modelo
model_id = "rubuntu/Phi-3.5-mini-instruct-Jopara-V3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

# ✅ Lista de exemplos para avaliação
avaliacao = [
    {
        "instr": "Traduzca al español:",
        "entrada": "Che sy oĩ che ndive.",
        "esperado": "Mi madre está conmigo."
    },
    {
        "instr": "Traduzca al guaraní:",
        "entrada": "¿Dónde está mi casa?",
        "esperado": "Moõpa oĩ che róga?"
    },
    {
        "instr": "Traduza para o Guarani:",
        "entrada": "¿Dónde está la escuela?",
        "esperado": "Moõpa oĩ mbo'ehao?"
    },
    {
        "instr": "Traducción al español:",
        "entrada": "Mba’éichapa nde réra?",
        "esperado": "¿Cómo te llamas?"
    }
]

# ✅ Função para gerar resposta
def gerar_resposta(instrucao, entrada):
    prompt = f"### Instrucción:\n{instrucao}\n\n### Entrada:\n{entrada}\n\n### Respuesta:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resposta.split("### Respuesta:")[-1].strip()

# ✅ Avaliação
resultados = []
for i, exemplo in enumerate(avaliacao, 1):
    gerado = gerar_resposta(exemplo["instr"], exemplo["entrada"])
    similaridade = fuzz.ratio(gerado.lower(), exemplo["esperado"].lower())
    resultados.append((i, exemplo["entrada"], gerado, exemplo["esperado"], similaridade))

# ✅ Exibir resultados
for r in resultados:
    print(f"--- Exemplo {r[0]} ---")
    print(f"Entrada: {r[1]}")
    print(f"Esperado: {r[3]}")
    print(f"Gerado: {r[2]}")
    print(f"Similaridade (fuzzy): {r[4]}%")
    print()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

--- Exemplo 1 ---
Entrada: Che sy oĩ che ndive.
Esperado: Mi madre está conmigo.
Gerado: 1444 o: "Oñemoha hína ojereve: Ñanemy: Ñembotũ mbo'a ha ñãjapy:'mỹ'mba'a 
 enefecio el problematojarí por
Similaridade (fuzzy): 20%

--- Exemplo 2 ---
Entrada: ¿Dónde está mi casa?
Esperado: Moõpa oĩ che róga?
Gerado: No está claro si tuendas ahora.

respuesta: "tendete" la"Juanfrí" alien"
dejaráo"la respuesta" ais": respuesta una"a. “

importará:
Similaridade (fuzzy): 13%

--- Exemplo 3 ---
Entrada: ¿Dónde está la escuela?
Esperado: Moõpa oĩ mbo'ehao?
Gerado: Házndo
un uniparce  con "one",""- como se tradría ( "Guía" en español es un sentido.

The two most Guess ( ").
I nación na? ndosingtéa.tova
Similaridade (fuzzy): 15%

--- Exemplo 4 ---
Entrada: Mba’éichapa nde réra?
Esperado: ¿Cómo te llamas?
Gerado: ha, yakupekó 🧬 Language,

Peteĩ, emperatry: ´## ndeikatu pysãi haʼe 
Ñeʼépe oguerekohá mbaʼapehýi
Similaridade (fuzzy): 14%

